In [ ]:
import xarray as xr
import pyproj
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import cmocean as cmo
from mpl_toolkits.axes_grid1 import make_axes_locatable
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import numpy as np
import copy

In [ ]:
run = 'fx03'
years = [1850]
#years = np.arange(1850,1860)

rho=1000
months = np.arange(1,13)

In [ ]:
#Prepare diagnostic variables
RO_A  = np.zeros((len(years),len(months)))
RO_O  = np.zeros((len(years),len(months)))

#Read generic NEMO data
ds = xr.open_dataset('../data/ecefiles/areas.nc')
area = ds['O1t0.srf'].values;
ds.close()
ds = xr.open_dataset(f'../data/ecefiles/{run}/{run}_1m_{years[0]}0101_{years[0]}1231_grid_T.nc')
lat = ds['nav_lat'].values
lon = ds['nav_lon'].values
time_bnds = ds['time_centered_bounds']
mask = np.where(lat<-58,1,0)
ds.close()

secs = 0.*months
for m,mm in enumerate(months):
    secs[m] = (time_bnds[m,1]-time_bnds[m,0]).values / np.timedelta64(1, 's')   

#Extract diagnostics
for y,yy in enumerate(years):
    ds = xr.open_dataset(f'../data/ecefiles/{run}/{run}_1m_{yy}0101_{yy}1231_grid_T.nc')
    rnf  = ds['friver'].values
    ds.close()
    for m,mm in enumerate(months):
        RO_O[y,m]  = np.nansum(rnf[m,:,:]*area*mask)*secs[m]*1e-12
    print(f'{yy}: RO_O: {np.sum(RO_O[y,:]):.0f} Gt/yr')

proj = ccrs.SouthPolarStereo()
cmap = copy.copy(mpl.cm.get_cmap("cmo.thermal"));
cmap.set_under('w')
cmap.set_bad('w')

RNF = np.nansum(rnf*area*mask*secs[:,np.newaxis,np.newaxis],axis=0)*1e-12

fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(111,projection=proj)
ax.set_extent([-280, 80, -80, -55], crs=ccrs.PlateCarree())
im = ax.scatter(lon,lat,5,c=RNF,cmap=cmap,transform=ccrs.PlateCarree(),lw=0,vmin=1e-10)
ax.coastlines(lw=.1)

gl = ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True)
gl.xlocator = mticker.FixedLocator(np.arange(-180,210,10))
gl.xformatter = LONGITUDE_FORMATTER

plt.colorbar(im);

In [ ]:
RNF4 = RNF.copy()

#Amun
RNF0 = np.where(np.logical_and(lon>-140,lon<-82),RNF,0)
RNF4 = np.where(RNF0>0,0,RNF4)
print('AMUN:',np.nansum(RNF0))

RNF1 = np.where(np.logical_or(lon>160,lon<200-360),RNF,0)
RNF4 = np.where(RNF1>0,0,RNF4)
print('ROSS:',np.nansum(RNF1))

RNF2 = np.where(np.logical_and(lon>60,lon<160),RNF,0)
RNF4 = np.where(RNF2>0,0,RNF4)
print('EAIS:',np.nansum(RNF2))

RNF3 = np.where(np.logical_and(np.logical_and(lon>-65,lon<-20),lat<-73),RNF,0)
RNF4 = np.where(RNF3>0,0,RNF4)
print('WEDD:',np.nansum(RNF3))

print('PENS:',np.nansum(RNF4))

print('TOT:',np.nansum(RNF0+RNF1+RNF2+RNF3+RNF4))

In [ ]:
fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(111,projection=proj)
ax.set_extent([-280, 80, -90, -63], crs=ccrs.PlateCarree())
im = ax.scatter(lon,lat,5,c=RNF0+RNF1+RNF2+RNF3+RNF4,cmap=cmap,transform=ccrs.PlateCarree(),lw=0,vmin=1e-10)
ax.coastlines(lw=.1)

gl = ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True)
gl.xlocator = mticker.FixedLocator(np.arange(-180,210,10))
gl.xformatter = LONGITUDE_FORMATTER

plt.colorbar(im);

In [ ]:
#Read generic IFS data
ds = xr.open_dataset('../data/ecefiles/areas.nc')
area = ds['A128.srf'].values
ds.close()
ds = xr.load_dataset(f"../data/ecefiles/{run}/ICMGG{run}+{years[0]}01",engine='cfgrib')
lon = ds['longitude'].values
lat = ds['latitude'].values
ds.close()
mask = np.where(lat<-60,1,0)

#Extract diagnostics
for y,yy in enumerate(years):
    for m,mm in enumerate(months):
        fname = f"../data/ecefiles/{run}/ICMGG{run}+{yy}{mm:02d}"
        ds = xr.load_dataset(fname,engine='cfgrib')
        time = ds['time'].values
        ro = ds['ro'].values
        ds.close()
        
        RO_A[y,m] = np.nansum(ro*mask[np.newaxis,:]*area[np.newaxis,:])*rho*1e-12

        #print(f'Got {yy} {mm}')
    print(f'{yy}: RO_A: {np.sum(RO_A[y,:]):.0f} Gt/yr')